In [ ]:
!pip install huggingface-hub optimum[onnxruntime]

In [ ]:
from huggingface_hub import snapshot_download

snapshot_download(
    repo_id="hooman650/bge-m3-onnx-o4",
    local_dir="bge-m3-onnx"
)

In [ ]:
from optimum.onnxruntime import ORTModelForFeatureExtraction
from transformers import AutoTokenizer
import torch

model = ORTModelForFeatureExtraction.from_pretrained(
    "bge-m3-onnx",
    provider="CPUExecutionProvider"
)

tokenizer = AutoTokenizer.from_pretrained("hooman650/bge-m3-onnx-o4")

In [34]:
sentences = [
    "English: The quick brown fox jumps over the lazy dog.",
    "Spanish: El rápido zorro marrón salta sobre el perro perezoso.",
    "French: Le renard brun rapide saute par-dessus le chien paresseux.",
    "German: Der schnelle braune Fuchs springt über den faulen Hund.",
    "Italian: La volpe marrone veloce salta sopra il cane pigro.",
    "Japanese: 速い茶色の狐が怠惰な犬を飛び越える。",
    "Chinese (Simplified): 快速的棕色狐狸跳过懒狗。",
    "Russian: Быстрая коричневая лиса прыгает через ленивую собаку.",
    "Arabic: الثعلب البني السريع يقفز فوق الكلب الكسول.",
    "Hindi: तेज़ भूरी लोमड़ी आलसी कुत्ते के ऊपर कूद जाती है।"
]

encoded_input = tokenizer(
    sentences,
    padding=True,
    truncation=True,
    return_tensors='pt'
)

# Get the embeddings
output = model(**encoded_input, return_dict=True).last_hidden_state

dense_vecs = torch.nn.functional.normalize(output[:, 0], dim=-1)

In [35]:
print(len(output))
print(len(output[0]))
print(len(output[0][0]))

10
24
1024


In [36]:
print(len(dense_vecs))
print(len(dense_vecs[0]))

10
1024
